In [2]:
!pip install transformers
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 12.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 21.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 767.5/767.5 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 MB 16.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 18.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 22.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 21.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 17.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6

In [23]:
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast

# 모델과 토크나이저 로드
model_name = "skt/kogpt2-base-v2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)

# 질문 문장 (원본 질문)
original_question = "서울은 대한민국의 수도인가요?"

# 프롬프트 설정 (질문을 다른 표현으로 바꾸도록 유도)
prompt = f"질문: {original_question}\n이 질문을 다른 표현으로 바꿔 주세요:\n새로운 질문:"

# 입력을 토크나이징
input_ids = tokenizer.encode(prompt, return_tensors='pt')

# 모델로부터 새로운 질문 생성
output = model.generate(input_ids, max_length=64, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)

# 생성된 문장을 디코딩
paraphrased_question = tokenizer.decode(output[0], skip_special_tokens=True)

print(f"Original Question: {original_question}")
print(f"Paraphrased Question: {paraphrased_question}")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


Original Question: 서울은 대한민국의 수도인가요?
Paraphrased Question: 질문: 서울은 대한민국의 수도인가요?
이 질문을 다른 표현으로 바꿔 주세요:
새로운 질문: 대한민국의 수도는 서울이 맞다고 생각하십니까?
이 질문에 대한 답은 다음과 같습니다.
서울은 수도라고 생각합니다.
서울이 수도이냐 아니냐를 따지는 것은 아닙니다.
서울의 수도, 서울의 수도라는


In [14]:
question_part = paraphrased_question.split("새로운 질문:")[1].split('\n')[0].strip()

print(f"Paraphrased Question: {question_part}")

Paraphrased Question: 대한민국의 수도는 서울이 맞다고 생각하십니까?


In [15]:
!pip install datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
import pandas as pd
from datasets import load_from_disk

dataset = load_from_disk("/data/ephemeral/home/jeongeun/data/raw/train_dataset")
train_dataset = dataset["train"]
train_df=pd.DataFrame(train_dataset)

def question2(question):
    prompt = f"질문: {question}\n이 질문을 다른 표현으로 바꿔 주세요:\n새로운 질문:"
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    output = model.generate(input_ids, max_length=64, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)

    paraphrased_question = tokenizer.decode(output[0], skip_special_tokens=True)
    question_part = paraphrased_question.split("새로운 질문:")[1].split('\n')[0].strip()

    return question_part

new_rows = train_df['question'].apply(question2)

# 원본 train_df에 새로운 행 추가
new_df = train_df.copy()
new_df['question'] = new_rows
train_df = pd.concat([train_df, new_df], ignore_index=True)

# 결과 확인
print(train_df[train_df['id'] == 'mrc-1-000067'])

      title                                            context  \
0     미국 상원  미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...   
3952  미국 상원  미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...   

                                               question            id  \
0                      대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?  mrc-1-000067   
3952  대통령, 부통령, 국가안보회의(NSC) 위원장, 국가정보국(DNI) 국장, NSC ...  mrc-1-000067   

                                      answers  document_id  __index_level_0__  
0     {'answer_start': [235], 'text': ['하원']}        18293                 42  
3952  {'answer_start': [235], 'text': ['하원']}        18293                 42  


In [30]:
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast

# 모델과 토크나이저 로드
model_name = "skt/kogpt2-base-v2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)

# 질문 문장 (원본 질문)
original_question = "대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?"

# 프롬프트 설정 (질문을 다른 표현으로 바꾸도록 유도)
prompt = f"질문: {original_question}\n이 질문을 다른 표현으로 바꿔 주세요:\n새로운 질문:"

# 입력을 토크나이징
input_ids = tokenizer.encode(prompt, return_tensors='pt')

# 모델로부터 새로운 질문 생성
output = model.generate(input_ids, max_length=256, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)

# 생성된 문장을 디코딩
paraphrased_question = tokenizer.decode(output[0], skip_special_tokens=True)

print(f"Original Question: {original_question}")
print(f"Paraphrased Question: {paraphrased_question}")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


Original Question: 대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?
Paraphrased Question: 질문: 대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?
이 질문을 다른 표현으로 바꿔 주세요:
새로운 질문: 대통령, 부통령, 국가안보회의(NSC) 위원장, 국가정보국(DNI) 국장, NSC 부보좌관, 국가안전보위부(NSA) 요원, NSA 국장 등등.
이 질문에 대한 답은 다음과 같습니다.
첫째, 대통령이나 부통령이 국가안보를 위해 어떤 일을 할 수 있는가?
둘째, 국가 안보를 위해서 무엇을 해야 하는가?
셋째, 국가를 위해 무엇을 할 것인가?
넷째, 국가의 안전을 위해 어떠한 일을 해야 하는가?
다섯째, 국가와 국민을 위해 무엇이 필요한가?
여섯째, 국가에 대한 국민의 신뢰를 회복하기 위해 어떻게 해야 할 것인가?
여덟째, 국민에 대한 국가의 의무를 이행하기 위해서 어떤 조치를 취해야 할 것인가를 묻는 질문이다.
이 질문은 다음과 같다.
대통령과 부통령은 국가의 안녕과 질서를 유지하기 위해 필요한 모든 조치를 취할 수 있다.
대통령은 국가 안보 및 안보에 관한 모든 권한을 행사한다.
대통령의 권한과 권한에 관한 사항은 대통령 또는 부통령에게 위임한다.
국가안보 및 안보회의 위원장은 대통령 및 부통령에 의해 임명된다.
국가안보와 관련된 모든 권한은 대통령(또는 부통령)에 의해 위임된다.
대통령이 권한을 행사할 수 있는 권한의 범위는 대통령(대통령)과 부통령(국무총리, 국방장관


In [1]:
%cd

/data/ephemeral/home
